# <center>KIỂM ĐỊNH ANOVA</center>

**ANOVA (Analysis of Variance)** có thể xem như là t-test mở rộng, nó được dùng để kiểm tra trung bình của nhiều hơn 2 nhóm có bằng nhau không?

$ H_0: \mu_1 = \mu_2 = ... = \mu_k $  
$ H_1: $ Các $\mu$ không bằng nhau  
k là số nhóm

# Điều kiện:
1. Normality: các nhóm phải có phân bố normal.
2. Homogeneity: các nhóm phải có variance như nhau.
3. Các nhóm phải độc lập với nhau

Khi các điều kiện trên không thoả, thì ta dùng **Kruskal-Wallis H-test or Welch’s ANOVA.**

Kết quả của ANOVA là dựa trên f distribution với 2 độ tự do df1=k-1, df2=n-k.

# Công thức của ANOVA là dựa trên mô hình linear regresion.

f-statistics $ = \frac{\text{(TSS-RSS)/df1}}{\text{RSS/df2}} $  
Trong đó:  
- TSS: total sum of square = $ \sum (y-\overline{y})^2 $
- RSS: residual sum of square = $ \sum (y-\hat{y})^2 $
- df1, df2: độ tự do của phân phối f, df1=k-1, df2=n-k
- n là số data, k là số nhóm

# Chỉ số eta-square với omega-square để ước tính hiệu quả của nghiên cứu với thực tế
eta-square tương đương với $R^2$ trong linear regression, tức là giải thích variance của model so với variance của data.

$\eta^2 = R^2 = \frac{\text{TSS - RSS}}{\text{TSS}} $

Omega-square là chỉ số tốt hơn eta-square vì nó không bias

<font size="+1">$ \omega^2 = \frac{\text{(TSS - RSS)} - \frac{\text{df1}}{\text{df2}} RSS   }{ \text{TSS} - \frac{1}{\text{df2}}\text{RSS} }$ </font>

# Kiểm tra điều kiện ANOVA bằng tests
Khi tính ANOVA thông qua linear regression thì các điều kiện lên data được áp dụng trên residual

1. Levene's test để kiểm tra *homogeneity of variance*
2. Shapiro's test để kiểm tra *normality*

# Post-hoc test
Kết quả anova cho phép ta nói biến số có hoặc không có liên quan outcome.  Nó không so sánh được giữa 2 nhóm trong đó với nhau. Do đó, ta cần post-hoc test.  

1. Bonferroni Correction Post-hoc Comparison  
Với f-stats, ta đặt mức có ý nghĩa là $\alpha$ là 0.05. Khi dùng Bonferroni, thì ta lấy $\alpha$ chia cho số lần so sánh là ra mức có ý nghĩa của t-test từng cặp nhóm.  
2. Tukey’s HSD Post-hoc comparison  
Tương tự như Bonferroni, tukey kiểm soát alpha chung của ANOVA = 0.05 bằng phương pháp khác, sau đó tính t-test sự khác nhau của từng cặp nhóm.

In [10]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)

df.head()

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4
